# Post Deployment Language Switching QA

[Task](https://phabricator.wikimedia.org/T275762)
|
[Schema](https://gerrit.wikimedia.org/r/plugins/gitiles/schemas/event/secondary/+/refs/heads/master/jsonschema/analytics/legacy/universallanguageselector/current.yaml)

# Contents
1. [Language Links in the Sidebar](#Language-Links-in-the-Sidebar)
2. [Language Switcher Button](#Language-Switcher-Button)

In [2]:
shhh <- function(expr) suppressPackageStartupMessages(suppressWarnings(suppressMessages(expr)))
shhh({
    library(tidyverse); 
    library(lubridate); 
    library(scales);
    library(magrittr); 
    library(dplyr)
})


## Language Links in the Siderbar

[Instrumentation Task](https://phabricator.wikimedia.org/T275762)

Instrumentation Notes:
- Instrumentation limited to legacy sidebar in modern Vector.
- Logged as `event.context = 'languages-list'`

In [2]:
query <- 
"
SELECT
    TO_DATE(dt) AS `date`,
    wiki,
    event.web_session_id,
    event.usereditbucket,
    event.timetochangelanguage,
    event.isanon,
    event.interfacelanguage,
    event.contentlanguage,
    event.selectedinterfacelanguage,
    Count(*) AS n_events
FROM event.universallanguageselector
WHERE
    year = 2021
    AND ((Month = 04 AND DAY > 26) OR (MONTH = 05))
    AND event.context = 'languages-list'
GROUP BY
    TO_DATE(dt),
    wiki,
    event.web_session_id,
    event.usereditbucket,
    event.timetochangelanguage,
    event.isanon,
    event.interfacelanguage,
    event.contentlanguage,
    event.selectedinterfacelanguage
"

In [3]:
lang_link_events <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [4]:
lang_link_events$date <- as.Date(lang_link_events$date)

## Daily Langauge Link Events

In [5]:
lang_link_events_daily <- lang_link_events %>%
    group_by(date) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

lang_link_events_daily

`summarise()` ungrouping output (override with `.groups` argument)



date,n_events,n_sessions
<date>,<int>,<int>
2021-04-28,791,636
2021-04-29,46934,33173
2021-04-30,274519,177001
2021-05-01,243709,151370
2021-05-02,258630,161650
2021-05-03,303583,195465
2021-05-04,300642,194931
2021-05-05,295520,191876
2021-05-06,291788,190268


We start recording events on 28 April 2021. There are an average 176,827 sessions per day including sessions by both logged in and logged out users. No unexpected spikes or drops so far.

## Clicks per session

In [ ]:
Check to make sure there are duplicate session id. Some sessions should have more than one click event.

In [9]:
length(unique(lang_link_events$web_session_id)) == nrow(lang_link_events)

[1] FALSE

In [6]:
head(lang_link_events)

,date,wiki,web_session_id,usereditbucket,timetochangelanguage,isanon,interfacelanguage,contentlanguage,selectedinterfacelanguage,n_events
,<date>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>
1,2021-04-28,dewikivoyage,1d774b99490d092ba4e4,100-999 edits,6208.225,false,de,de,it,1
2,2021-04-28,dewikivoyage,95f1f7824782d3b0a91b,NULL,98815.000,true,de,de,en,1
3,2021-04-28,dewikivoyage,e06266b83d26d1865339,NULL,128051.000,true,de,de,en,1
4,2021-04-28,frwiktionary,082c55c27c55a4860c58,NULL,8196.865,true,fr,fr,en,1
5,2021-04-28,frwiktionary,18ecbc928fb6bfc5df5e,NULL,5913.570,true,fr,fr,pt,1
6,2021-04-28,frwiktionary,1dbcf88e81ed2a100b27,NULL,40165.625,true,fr,fr,en,1


In [12]:
lang_link_events_persession <- lang_link_events %>%
    group_by(isanon) %>%
    summarize(avg_clicks = mean(n_events),
              max_clicks = max(n_events),
              min_clciks = min(n_events))

lang_link_events_persession

`summarise()` ungrouping output (override with `.groups` argument)



isanon,avg_clicks,max_clicks,min_clciks
<chr>,<dbl>,<int>,<int>
false,1.000327,2,1
true,1.000169,4,1


## By Logged In Status

In [13]:
lang_link_events_isanon <- lang_link_events %>%
    group_by(isanon) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

lang_link_events_isanon 

`summarise()` ungrouping output (override with `.groups` argument)



isanon,n_events,n_sessions
<chr>,<int>,<int>
false,113161,46278
true,3568719,2058676


97.8% of all sessions with clicks to the language links are by logged out users. That's high but expected because instrumentation was limited to legacy sidebar in modern Vector (not from legacy or other skins such as timeless). The new language switching functionality was made available to all logged-in users opted into the latest version of the Vector skin.

Legacy sidebar in modern Vector would mostly appear to logged-out users on test wikis where Vector is deployed as default.


## By Test Wiki

In [36]:
# events and sessions that include link to language link by test wiki
lang_link_events_testwiki <- lang_link_events %>%
    filter(wiki %in% c('frwiktionary', 'hewiki', 'ptwikiversity', 'frwiki', 
    'euwiki', 'fawiki', 'ptwiki', 'kowiki', 'trwiki', 'srwiki', 'bnwiki', 'dewikivoyage', 'vecwiki' )) %>%
    group_by(wiki) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

lang_link_events_testwiki


`summarise()` ungrouping output (override with `.groups` argument)



wiki,n_events,n_sessions
<chr>,<int>,<int>
bnwiki,8451,5726
dewikivoyage,755,604
euwiki,31609,22351
fawiki,200781,110092
frwiki,1933743,1091442
frwiktionary,21307,13350
hewiki,173526,100165
kowiki,146461,81522
ptwiki,555228,328675


In [39]:
# events and sessions that include link to language link by test wiki category and logged-in status
lang_link_events_testwiki_isanon <- lang_link_events %>%
    mutate(istestwiki = ifelse(wiki %in% c('frwiktionary', 'hewiki', 'ptwikiversity', 'frwiki', 
    'euwiki', 'fawiki', 'ptwiki', 'kowiki', 'trwiki', 'srwiki', 'bnwiki', 'dewikivoyage', 'vecwiki' ), 'test_wiki', 'non_test_wiki')) %>%
    group_by(istestwiki, isanon) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

lang_link_events_testwiki_isanon

`summarise()` regrouping output by 'istestwiki' (override with `.groups` argument)



istestwiki,isanon,n_events,n_sessions
<chr>,<chr>,<int>,<int>
non_test_wiki,false,3540,1422
non_test_wiki,true,147,80
test_wiki,false,102371,41836
test_wiki,true,3325559,1922334


Almost all of the events recorded to date (99%) have been on test wikis. This is expected as the new language switcher button was deployed to all users opt'd in to the modern vector on all non test wikis. Users with modern vector on the test wikis have not been shown the new language switcher and still shown the language links in the sidebar.

On non test wikis, the majority (94.67%) of sessions with clicks to the language list on modern vector come from logged-in users. Need to confirm if it's possible to have language link in sidebar if you are logged-in, on modern vector and on a non test wiki. 


## By User Edit Bucket

In [30]:
logged_in_editcount <- lang_link_events %>%
    filter(isanon == 'false') %>%
    group_by(usereditbucket) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

logged_in_editcount

`summarise()` ungrouping output (override with `.groups` argument)



usereditbucket,n_events,n_sessions
<chr>,<int>,<int>
0 edits,20092,10961
1-4 edits,11510,5643
100-999 edits,17264,6533
1000+ edits,31482,8932
5-99 edits,25556,11332
NULL,7,5


In [31]:
logged_out_editcount <- lang_link_events %>%
    filter(isanon == 'true') %>%
    group_by(usereditbucket) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

logged_out_editcount

`summarise()` ungrouping output (override with `.groups` argument)



usereditbucket,n_events,n_sessions
<chr>,<int>,<int>
5-99 edits,1,1
NULL,3325705,1922412


There are just a few instances (under 0.01%) of the event.usereditbucket field being populated for logged out users and recorded as NULL for logged-in users. Further investigation might be needed; however, the numbers of these events is not high enough to skew the data.

## By Final Language

In [43]:
# test that you can switch from one language to the next.
top_final_languages <- lang_link_events %>%
    mutate(all_sessions = n_distinct(web_session_id)) %>%
    group_by(selectedinterfacelanguage) %>%
    summarize(n_sessions = n_distinct(web_session_id),
             pct_sessions = n_sessions/all_sessions) %>%
    distinct() %>%
    arrange(desc(n_sessions))

head(top_final_languages )

`summarise()` regrouping output by 'selectedinterfacelanguage' (override with `.groups` argument)



selectedinterfacelanguage,n_sessions,pct_sessions
<chr>,<int>,<dbl>
en,1443105,0.73470332
es,128358,0.06534871
de,125061,0.06367016
it,65010,0.03309743
ar,64408,0.03279094
ru,52428,0.02669177


The most frequent language switches are to english (73% of sessions) followed by spanish (6.5%), and german (6.3%).

## By Initial Language 

In [48]:

top_initial_languages <- lang_link_events %>%
    mutate(all_sessions = n_distinct(web_session_id)) %>%
    group_by(interfacelanguage, contentlanguage) %>%
    summarize(n_sessions = n_distinct(web_session_id),
             pct_sessions = n_sessions/all_sessions) %>%
    distinct() %>%
    arrange(desc(n_sessions))

head(top_initial_languages )

`summarise()` regrouping output by 'interfacelanguage', 'contentlanguage' (override with `.groups` argument)



interfacelanguage,contentlanguage,n_sessions,pct_sessions
<chr>,<chr>,<int>,<dbl>
fr,fr,1103742,0.56192925
pt,pt,328012,0.16699513
tr,tr,150977,0.07686433
fa,fa,109989,0.05599681
he,he,100125,0.05097493
ko,ko,81429,0.04145655


The interfacelanguage and contentlanguage will usually be the same and should match for most instances, which is confirmed here.

The top initial languages all from test wikis, which is expected since the language links are still shown to all logged-in and logged-out users on modern vector. The new language switcher, which replaces the lang links with a button, are show to all logged-in users opt'd into modern vector on non-test wikis. 

In [ ]:
## Most Frequent Switch Types


In [49]:
top_final_languages <- lang_link_events %>%
    mutate(all_sessions = n_distinct(web_session_id)) %>%
    group_by(interfacelanguage, selectedinterfacelanguage) %>%
    summarize(n_sessions = n_distinct(web_session_id),
             pct_sessions = n_sessions/all_sessions) %>%
    distinct() %>%
    arrange(desc(n_sessions))

head(top_final_languages )

`summarise()` regrouping output by 'interfacelanguage', 'selectedinterfacelanguage' (override with `.groups` argument)



interfacelanguage,selectedinterfacelanguage,n_sessions,pct_sessions
<chr>,<chr>,<int>,<dbl>
fr,en,775769,0.39495398
pt,en,264373,0.13459570
tr,en,118824,0.06049483
fa,en,95622,0.04868239
fr,de,94915,0.04832245
he,en,84668,0.04310557


39% of all sessions are switches are from French to English. 

# Language Switcher Button

[Instrumentation Task](https://phabricator.wikimedia.org/T281928)

- Instrumented as `event.action="compact-language-links-open"`
- Logged when a user open the new language switcher button. 
- The first click and subsequent clicks should be recorded per https://gerrit.wikimedia.org/r/c/mediawiki/extensions/UniversalLanguageSelector/+/688994/
- Change merged on May 11th and backported/deployed on May 12th.
- There should not be any clicks to the Language Switcher button by logged-in users test wikis, as the new lanuage button has not been deployed to these wikis. 

In [72]:
query <- 
"
SELECT
    TO_DATE(dt) AS `date`,
    wiki,
    event.web_session_id,
    event.usereditbucket,
    event.timetochangelanguage,
    event.isanon,
    event.context,
    event.interfacelanguage,
    event.contentlanguage,
    event.selectedinterfacelanguage,
    Count(*) AS n_events
FROM event.universallanguageselector
WHERE
    year = 2021
    AND month = 05
    AND event.action = 'compact-language-links-open'
    AND useragent.is_bot = false
GROUP BY
    TO_DATE(dt),
    wiki,
    event.web_session_id,
    event.usereditbucket,
    event.timetochangelanguage,
    event.isanon,
    event.context,
    event.interfacelanguage,
    event.contentlanguage,
    event.selectedinterfacelanguage
"

In [73]:
lang_button_events <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



## Daily Lang Button Click Events

In [30]:
lang_button_events_daily <- lang_button_events %>%
    group_by(date) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

lang_button_events_daily

`summarise()` ungrouping output (override with `.groups` argument)



date,n_events,n_sessions
<chr>,<int>,<int>
2021-05-12,12,10
2021-05-13,194,159
2021-05-14,1317,1147
2021-05-15,1159,928
2021-05-16,1358,1042
2021-05-17,32668,28541
2021-05-18,90811,78945
2021-05-19,89299,77108
2021-05-20,87644,75820


- Confirmed we start recording events on May 12th following backport of the change, starting with just a few events and then more as change rolled out to additional wikis.
- Since deployment through today (24-May 20201), we've logged  at total of 508,077 sessions with a click to the language button and 600,955 clicks to the the language button. There is an expected higher number of events than sessions, which is expected as a session can include multiple clicks to open the language button.


## Double Check to make sure there are some sessions with more than one event

In [31]:
length(unique(lang_button_events$web_session_id)) == nrow(lang_button_events)

[1] FALSE

## Lang Button Events Per Wiki

In [32]:

lang_button_events_bywiki <- lang_button_events %>%
    group_by(wiki) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id)) %>%
    arrange(desc(n_sessions))

head(lang_button_events_bywiki)

`summarise()` ungrouping output (override with `.groups` argument)



wiki,n_events,n_sessions
<chr>,<int>,<int>
enwiki,309814,252578
ruwiki,43849,35509
dewiki,36597,30870
eswiki,29366,24740
frwiki,25537,21115
zhwiki,18207,15367


Almost half (49.7%) of sessions with a click to the language button were recorded on English Wikipedia, followed by Russian, German and Spanish Wikipedia. Note: The language switcher button is currently not availabe to logged-in users on test wikis.



# Lang Button Event on Test Wikis

In [33]:

lang_button_events_bytestwiki <- lang_button_events %>%
    filter(wiki %in% c('frwiktionary', 'hewiki', 'ptwikiversity', 'frwiki', 
    'euwiki', 'fawiki', 'ptwiki', 'kowiki', 'trwiki', 'srwiki', 'bnwiki', 'dewikivoyage', 'vecwiki' ))  %>%
    group_by(wiki) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id)) %>%
    arrange(desc(n_sessions))

lang_button_events_bytestwiki

`summarise()` ungrouping output (override with `.groups` argument)



wiki,n_events,n_sessions
<chr>,<int>,<int>
frwiki,25537,21115
ptwiki,6665,5682
trwiki,5967,5052
srwiki,3898,3455
kowiki,2608,2158
fawiki,2494,2055
hewiki,2325,1992
euwiki,536,482
frwiktionary,506,445


About 8.4% of all sessions were recorded on the early adopter wikis. 

I reviewed the number of sessions on test wikis by logged-in status to determine if these were mostly logged-out or logged-in users.

In [34]:
# events and sessions that include link to language link by test wiki category and logged-in status
lang_button_events_testwiki_isanon <- lang_button_events %>%
    mutate(istestwiki = ifelse(wiki %in% c('frwiktionary', 'hewiki', 'ptwikiversity', 'frwiki', 
    'euwiki', 'fawiki', 'ptwiki', 'kowiki', 'trwiki', 'srwiki', 'bnwiki', 'dewikivoyage', 'vecwiki' ), 'test_wiki', 'non_test_wiki')) %>%
    group_by(istestwiki, isanon) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

lang_button_events_testwiki_isanon 

`summarise()` regrouping output by 'istestwiki' (override with `.groups` argument)



istestwiki,isanon,n_events,n_sessions
<chr>,<chr>,<int>,<int>
non_test_wiki,false,20053,12641
non_test_wiki,true,529967,442145
test_wiki,false,1951,1478
test_wiki,true,48984,41332


0.29% of sessions with a click to the lang switcher button came from logged-in users on the early adopter wikis and 8.1% of sessions by logged-out users. 

NEED TO CONFIRM IF THIS IS EXPECTED.

## By anon status

In [74]:
lang_button_events_byanon <- lang_button_events %>%
    group_by(isanon) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id)) 

head(lang_button_events_byanon)

`summarise()` ungrouping output (override with `.groups` argument)



isanon,n_events,n_sessions
<chr>,<int>,<int>
false,22534,14454
true,592126,494268


The majority of sessions with clicks to the language button are by logged out users (97.2%) 

## Clicks per session

In [35]:
lang_button_events_persession <- lang_button_events %>%
    group_by(isanon) %>%
    summarize(avg_clicks = mean(n_events),
              max_clicks = max(n_events),
              min_clciks = min(n_events))

lang_button_events_persession

`summarise()` ungrouping output (override with `.groups` argument)



isanon,avg_clicks,max_clicks,min_clciks
<chr>,<dbl>,<int>,<int>
false,1.461477,49,1
true,1.173652,754,1


Most sessions include between 1 to 2 clicks to the language switcher button.

Note: There are some sessions by anon users with over 600 clicks, which is is likely automated traffic from bots and can be filtered out.

## User Edit Bucket

In [36]:
logged_in_editcount <- lang_button_events %>%
    filter(isanon == 'false') %>%
    group_by(usereditbucket) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

logged_in_editcount

`summarise()` ungrouping output (override with `.groups` argument)



usereditbucket,n_events,n_sessions
<chr>,<int>,<int>
0 edits,5936,4666
1-4 edits,2404,1764
100-999 edits,3215,1888
1000+ edits,5293,2628
5-99 edits,5156,3202


In [37]:
logged_out_editcount <- lang_button_events %>%
    filter(isanon == 'true') %>%
    group_by(usereditbucket) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

logged_out_editcount

`summarise()` ungrouping output (override with `.groups` argument)



usereditbucket,n_events,n_sessions
<chr>,<int>,<int>
NULL,578951,483477


Confirmed that we are not recording an edit count for logged out users as expected.

## Content Language

In [38]:

top_content_languages <- lang_button_events %>%
    mutate(all_sessions = n_distinct(web_session_id)) %>%
    group_by(interfacelanguage, contentlanguage) %>%
    summarize(n_sessions = n_distinct(web_session_id),
             pct_sessions = n_sessions/all_sessions) %>%
    distinct() %>%
    arrange(desc(n_sessions))

head(top_content_languages )

`summarise()` regrouping output by 'interfacelanguage', 'contentlanguage' (override with `.groups` argument)



interfacelanguage,contentlanguage,n_sessions,pct_sessions
<chr>,<chr>,<int>,<dbl>
en,en,257528,0.51773060
ru,ru,35846,0.07206428
de,de,31134,0.06259135
es,es,24863,0.04998422
fr,fr,21550,0.04332381
it,it,13693,0.02752821


The top current content and interface lang settings are english, russian, german and spanish, which fits with the top wikis we see where the language button switch clicks occur.

## Event Context

In [39]:

lang_button_events_context <- lang_button_events %>%
    group_by(context) %>%
    summarize(n_events = sum(n_events),
             n_sessions = n_distinct(web_session_id))

lang_button_events_context

`summarise()` ungrouping output (override with `.groups` argument)



context,n_events,n_sessions
<chr>,<int>,<int>
NULL,600955,497417


We are not recording `event.context` for these events as expected. 

## Time to Change Language

In [40]:

lang_button_events_time <- lang_button_events %>%
    summarise(avg_time = mean(timetochangelanguage),
             median_time = median(timetochangelanguage),
             max_time = max(timetochangelanguage),
             min_time = min(timetochangelanguage))

lang_button_events_time

Warning message in mean.default(timetochangelanguage):
“argument is not numeric or logical: returning NA”
Warning message in mean.default(sort(x, partial = half + 0L:1L)[half + 0L:1L]):
“argument is not numeric or logical: returning NA”


avg_time,median_time,max_time,min_time
<dbl>,<dbl>,<chr>,<chr>
NA,NA,NULL,NULL


Note: We don't record time to change language for the initial language but we record when the user actually changes languages.

## Confirm we can identify language switches for sessions where button is clicked

In [77]:
#rough query to confirm approach
# will be refine in analysis
query <- 
"
-- sessions where lang button was selected
WITH button AS (
SELECT
    MIN(TO_DATE(dt)) as button_date,
    event.web_session_id as session_id,
    wiki as wiki
FROM event.universallanguageselector
WHERE
    year = 2021
    AND month = 05
    AND Day >= 12
    AND useragent.is_bot = false
    AND event.action = 'compact-language-links-open'
GROUP BY 
    event.web_session_id,
    wiki
),

lang_switches AS (
  SELECT
    TO_DATE(dt) as switch_date,
    event.web_session_id as session_id,
    event.context as switch_context,
    wiki as wiki
FROM event.universallanguageselector
WHERE
    year = 2021
    AND month = 05
    AND Day >= 12
    AND useragent.is_bot = false
    AND event.action = 'language-change'
)

SELECT
    button.button_date,
    lang_switches.switch_date,
    button.session_id,
    button.wiki,
-- sessions with lang switch that occured after button clicks
    IF(lang_switches.session_id IS NOT NULL AND switch_date >= button_date, 1, 0) AS language_switch,
    lang_switches.switch_context
FROM button
LEFT JOIN lang_switches ON
    button.session_id = lang_switches.session_id AND
    button.wiki = lang_switches.wiki
    
"

In [78]:
lang_button_switch_events <-  wmfdata::query_hive(query)

Don't forget to authenticate with Kerberos using kinit



In [79]:
#reformat to date format
lang_button_switch_events$button_date <- as.Date(lang_button_switch_events$button_date, format = "%Y-%m-%d")
lang_button_switch_events$switch_date <- as.Date(lang_button_switch_events$switch_date, format = "%Y-%m-%d")

## Lang Button Sessions with Language Switch

In [80]:
lang_button_switch_events_nsessions <- lang_button_switch_events %>%
    mutate(language_switch = ifelse(language_switch == 0, "no_switch", "switch")) %>%
    group_by(language_switch) %>%
    summarise(n_sessions = n_distinct(session_id))

lang_button_switch_events_nsessions

`summarise()` ungrouping output (override with `.groups` argument)



language_switch,n_sessions
<chr>,<int>
no_switch,495948
switch,17450


About 3.4% of sessions where the language button was clicked was followed by an event to switch the language. 

## Lang Button Sessions with Switch by Context

In [69]:
lang_button_switch_events_nsessions <- lang_button_switch_events %>%
    filter(language_switch == 1) %>%
    group_by(switch_context) %>%
    summarise(n_sessions = n_distinct(session_id))

lang_button_switch_events_nsessions

`summarise()` ungrouping output (override with `.groups` argument)



switch_context,n_sessions
<chr>,<int>
interface,156
languages-list,17049


Confirmed that we are only recording `event.context` for langauge switches. There are two types of switches we are recording: 'interface" and 'languages-list'.

Almost all (99%) of sessions that switched languages after clicking the language button were recorded with `event.context = 'languages-list'`.  

Note: We also use 'languages-list' to identify clicks to the sidebar. I'll need to use sessions that opened language switcher button to identify those that recieved the new treatment.